<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# Regularization Exercise

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Run this before any other code cell
# This downloads the csv data files into the same directory where you have saved this notebook

import urllib.request
from pathlib import Path
import os
path = Path()

# Dictionary of file names and download links
files = {'Auto.csv':'https://storage.googleapis.com/aipi_datasets/Auto.csv'}

# Download each file
for key,value in files.items():
    filename = path/key
    url = value
    # If the file does not already exist in the directory, download it
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url,filename)

In [ ]:
def load_data(filename):
    # Read in data
    data = pd.read_csv(filename)
    # Remove rows with missing values
    data = data[data['horsepower'] != '?'].copy()
    return data

data = load_data('Auto.csv')
data.head()

## LASSO Regression
Let's use LASSO Regression to add regularization and see what impact it has on the features included in the model.  First we will run a standard linear regression to get a baseline.

In [ ]:
def prep_data_allfeats(data,pct):
    # Define the features and response (X and y)
    X = data[['cylinders','displacement','horsepower','weight','acceleration','year']].copy().astype(int)
    y = data['mpg'].copy().astype(float)

    # Split into training and test sets
    X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=0,test_size=pct)
    return X_train,X_test,y_train,y_test

# Split our data
X_train,X_test,y_train,y_test = prep_data_allfeats(data,pct=0.2)

Now let's try a LASSO model.  First we need to scale our data

In [ ]:
def train_lasso(X_train,y_train,alpha=1.):
    # First we scale our data - remember, only use the training data to fit the scaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Run a LASSO model
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train_scaled,y_train)
    return scaler,lasso_model

def test_model(model, X_test, y_test,transform=None):
    # Compute the MSE for a model
    if transform is not None:
        X_test = transform.transform(X_test)
    preds = model.predict(X_test)
    mse = 1/len(y_test)*np.sum((preds-y_test)**2)
    return mse
    
scaler,model = train_lasso(X_train,y_train)
test_mse = test_model(model, X_test, y_test,transform=scaler)
print('MSE on the test set is {:.3f}'.format(test_mse))

# Display the equation for the LASSO model
coef = model.coef_
intercept = model.intercept_
equation = 'y = {:.3f} + '.format(intercept) + ' + '.join(['{:.3f}*{}'.format(coef,var) for coef,var in zip(coef,X_train.columns)])
print(equation)

# Plot the coefficients
plt.barh(X_train.columns,coef)
plt.axvline(x=0, color='.5')
plt.show()

As we can see above, our LASSO model at lambda/alpha=1.0 zeroed out several of our model coefficients, leaving only weight, horsepower, and year in our model.

Let's look for the optimal value of the lambda (alpha) hyperparameter.  Since the possible range of values for lambda is 0 to infinity, let's start with a big picture analysis and then zoom in

In [ ]:
def optimize_regularization(X_train,y_train,model,alpha_vals,nsplits=10):
    '''
    Finds the optimal value of alpha/lambda from a given range using cross-validation

    Inputs:
        X_train(pd.DataFrame): training set inputs
        y_train(pd.Series): training set labels
        model(sklearn.base.BaseEstimator): instantiated scikit-learn model object
        alpha_vals(list): list of values to evaluate for alpha/lambda
        nsplits(int): number of folds for cross-validation

    Returns:
        opt_lambda(float): alpha/lambda value from the input list that results in best cross-validation performance
        errors(list): list of mean MSE across validation folds for each value of alpha/lambda
    '''

    ### BEGIN SOLUTION ####

    

    ### END SOLUTION ###


# Now let's vary the lambda (alpha) hyperparameter and find the optimal value using cross-validation
alpha_vals = [10**i for i in range(-3,3,1)]
model = Lasso()
opt_alpha, errors = optimize_regularization(X_train,y_train,model,alpha_vals,nsplits=10)
assert len(errors) == len(alpha_vals)
assert opt_alpha == 0.1
    
# Plot the mse vs lambda/alpha values, using log scale for lambda values
plt.plot(np.log(alpha_vals),errors)
plt.xticks(ticks=np.log(alpha_vals),labels=alpha_vals)
plt.xlabel('lambda')
plt.ylabel('mse')
plt.show()

Now let's zoom in to identify the optimal lambda value

In [ ]:
alpha_vals = np.arange(0.,1.01,0.01)
opt_alpha, errors = optimize_regularization(X_train,y_train,model,alpha_vals,nsplits=10)

print('Optimal lambda value is {:.3f}'.format(opt_alpha))
    
# Plot the mse vs lambda/alpha values
plt.plot(alpha_vals,errors)
plt.xlabel('lambda')
plt.ylabel('mse')
plt.show()

### Based on what we learned in the lecture, why might higher penalty values for the LASSO not work well in this particular case?

In [ ]:
# Now let's vary the lambda (alpha) hyperparameter and visualize the change in coefficients
coeff_vals = {var:[] for var in X_train.columns} # Dict to hold values of each coefficient

alpha_vals = np.arange(0.,5.01,0.1)
for val in alpha_vals:
    scaler,lasso_model = train_lasso(X_train,y_train,alpha=val)
    coef = lasso_model.coef_
    for coef,var in zip(coef,X_train.columns):
        coeff_vals[var].append(coef)
        
# Plot the coefficient values
plt.figure(figsize=(8,8))
for var in coeff_vals.keys():
    plt.plot(alpha_vals,coeff_vals[var],label=var)
plt.plot([0,np.max(alpha_vals)],[0,0],color='black')
plt.xlim(0,np.max(alpha_vals))
plt.legend()
plt.xlabel('lambda')
plt.show()


In [ ]:
# Evaluate performance of the model with the optimized alpha/lambda
scaler,model = train_lasso(X_train,y_train,alpha=opt_alpha)
test_mse = test_model(model, X_test, y_test, transform=scaler)
print('MSE on the test set is {:.3f}'.format(test_mse))

# Display the equation for the LASSO model
coef = model.coef_
intercept = model.intercept_
equation = 'y = {:.3f} + '.format(intercept) + ' + '.join(['{:.3f}*{}'.format(coef,var) for coef,var in zip(coef,X_train.columns)])
print(equation)

# Plot the coefficients
plt.barh(X_train.columns,coef)
plt.axvline(x=0, color='.5')
plt.show()

## Ridge Regression

In [ ]:
def train_ridge(X_train,y_train,alpha=1.0):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Run a Ridge model using the default lambda (alpha)
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train_scaled,y_train)
    
    return scaler, ridge_model

scaler,model = train_ridge(X_train,y_train)
test_mse = test_model(model, X_test, y_test, transform=scaler)
print('MSE on the test set is {:.3f}'.format(test_mse))

In [ ]:
# Now let's vary the lambda (alpha) hyperparameter and find the optimal value using cross-validation
alpha_vals = [10**i for i in range(-3,3,1)]
model = Ridge()
opt_alpha_ridge, errors = optimize_regularization(X_train,y_train,model,alpha_vals,nsplits=10)
print('Of the values evaluated, the optimal lambda value is {:.3f}'.format(opt_alpha_ridge))
    
# Plot the mse vs lambda/alpha values
plt.plot(np.log(alpha_vals),errors)
plt.xticks(ticks=np.log(alpha_vals),labels=alpha_vals)
plt.xlabel('lambda')
plt.ylabel('mse')
plt.show()

In [ ]:
# Now let's vary the lambda (alpha) hyperparameter and find the optimal value using cross-validation
alpha_vals = np.arange(0.,5.01,0.1)
model = Ridge()
opt_alpha_ridge, errors = optimize_regularization(X_train,y_train,model,alpha_vals,nsplits=10)
print('Of the values evaluated, the optimal lambda value is {:.3f}'.format(opt_alpha_ridge))
    
# Plot the mse vs lambda/alpha values
plt.plot(alpha_vals,errors)
plt.xlabel('lambda')
plt.ylabel('mse')
plt.show()

In [ ]:
# Now let's vary the lambda (alpha) hyperparameter and visualize the change in coefficients
coeff_vals = {var:[] for var in X_train.columns} # Dict to hold values of each coefficient

alpha_vals = np.arange(0.,5.05,0.05)
for val in alpha_vals:
    scaler,model = train_ridge(X_train,y_train,val)
    # Get coefficients to plot
    coef = model.coef_
    for coef,var in zip(coef,X_train.columns):
        coeff_vals[var].append(coef)
        
# Plot the coefficient values
plt.figure(figsize=(8,8))
for var in coeff_vals.keys():
    plt.plot(alpha_vals,coeff_vals[var],label=var)
plt.plot([0,np.max(alpha_vals)],[0,0],color='black')
plt.xlim(0,np.max(alpha_vals))
plt.legend()
plt.xlabel('lambda')
plt.show()

In [ ]:
# Evaluate performance of the model with the optimized alpha/lambda
scaler,model = train_ridge(X_train,y_train,alpha=opt_alpha)
test_mse = test_model(model, X_test, y_test, transform=scaler)
print('MSE on the test set is {:.3f}'.format(test_mse))

# Display the equation for the LASSO model
coef = model.coef_
intercept = model.intercept_
equation = 'y = {:.3f} + '.format(intercept) + ' + '.join(['{:.3f}*{}'.format(coef,var) for coef,var in zip(coef,X_train.columns)])
print(equation)

# Plot the coefficients
plt.barh(X_train.columns,coef)
plt.axvline(x=0, color='.5')
plt.show()